In [1]:
# Append location to path to allow custom modules to be used.
import sys, os
sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import cgael
from cgael.models.SimpleColor import *

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer

import pygad
import pygad.kerasga

import numpy as np
import pandas as pd

from PIL import Image

In [3]:
ts = cgael.LanguageTokenSet("CHAT", '-')

In [4]:
msg1 = ts.encode("CATCH THAT CAT", shape=(4,5))
msg1

<tf.Tensor: shape=(4, 5), dtype=int64, numpy=
array([[1, 3, 4, 1, 2],
       [4, 2, 3, 4, 0],
       [1, 3, 4, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)>

In [5]:
msg2 = ts.encode("", shape=(4,5))

In [6]:
msg3 = ts.encode("CHAT AT HATCH HAT", shape=(4,5))
msg3

<tf.Tensor: shape=(4, 5), dtype=int64, numpy=
array([[1, 2, 3, 4, 0],
       [3, 4, 0, 0, 0],
       [2, 3, 4, 1, 2],
       [2, 3, 4, 0, 0]], dtype=int64)>

In [31]:
msg4 = ts.encode("ATATA TATAT ATATA TATAT", shape=(4,5))
msg4

<tf.Tensor: shape=(4, 5), dtype=int64, numpy=
array([[3, 4, 3, 4, 3],
       [4, 3, 4, 3, 4],
       [3, 4, 3, 4, 3],
       [4, 3, 4, 3, 4]], dtype=int64)>

In [34]:
batch = tf.convert_to_tensor([msg1, msg2, msg3, msg4])
batch

<tf.Tensor: shape=(4, 4, 5), dtype=int64, numpy=
array([[[1, 3, 4, 1, 2],
        [4, 2, 3, 4, 0],
        [1, 3, 4, 0, 0],
        [0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 2, 3, 4, 0],
        [3, 4, 0, 0, 0],
        [2, 3, 4, 1, 2],
        [2, 3, 4, 0, 0]],

       [[3, 4, 3, 4, 3],
        [4, 3, 4, 3, 4],
        [3, 4, 3, 4, 3],
        [4, 3, 4, 3, 4]]], dtype=int64)>

In [8]:
from cgael.metrics import brevity

In [14]:
brevity.simple_brevity(msg1, power=1.2)

<tf.Tensor: shape=(), dtype=float64, numpy=0.48801731075147936>

In [15]:
msg1

<tf.Tensor: shape=(4, 5), dtype=int64, numpy=
array([[1, 3, 4, 1, 2],
       [4, 2, 3, 4, 0],
       [1, 3, 4, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)>

In [50]:
def power_brevity(data, word_length_power=2, word_count_power=2):
    @tf.function
    def helper(mask):
        # STEP 1: WORD LENGTH POWER
        # Get the lengths of each word.
        x = tf.math.reduce_sum(mask, axis=-1)
        # Divide by maximum length of words, placing the function on the range [0, 1].
        x = tf.math.divide(x, data.shape[-1])
        # Apply word_length_power.
        x = tf.math.pow(x, word_length_power)
        
        # STEP 2: WORD COUNT POWER
        # Get the sum of each word score.
        x = tf.math.reduce_sum(x, axis=-1)
        # Divide by maximum number of words, placing the function on the range [0, 1].
        x = tf.math.divide(x, data.shape[-2])
        # Apply word_count_power.
        x = tf.math.pow(x, word_count_power)
    
        return x
    
    # Get binary mask of data.
    mask = tf.sign(data)
    sums = tf.math.reduce_sum(mask, axis=[-2, -1])
    results = tf.where(
        condition = tf.math.equal(sums, 0),
        x = tf.constant(1, dtype=tf.float64),
        y = helper(mask)
    )
    return tf.reduce_prod(results)
    
power_brevity(msg4, word_count_power=3)

<tf.Tensor: shape=(), dtype=float64, numpy=1.0>

---

In [ ]:
lengths = tf.math.reduce_sum(tf.math.sign(batch), axis=-1)
lengths

In [ ]:
scores = tf.math.reduce_sum(tf.math.reduce_sum(tf.math.sign(batch), axis=-1), axis=-1)
scores

In [ ]:
totals = tf.math.reduce_sum(tf.math.sign(batch))
totals

In [ ]:
area = tf.math.reduce_sum(tf.math.reduce_prod(batch.shape))
area

In [ ]:
batch = msg1

In [ ]:
b = tf.math.reduce_prod(tf.constant(batch.shape, dtype=tf.int64))
e = tf.math.reduce_sum(tf.math.sign(batch), axis=[-2, -1])
f = tf.math.equal(e, 0)
g = batch.shape[-2]*batch.shape[-1]
h = tf.where(f, g, tf.math.subtract(e,1))
i = tf.math.reduce_sum(h)
j = tf.math.divide(i, b)
print(j)

In [ ]:
def simple_brevity(data):
    # Get the number of non-padded tokens for each entry of the batch.
    sums = tf.math.reduce_sum(tf.math.sign(data), axis=[-2, -1])
    # Calculate the score of each entry of the batch 'n' such that:
    # - if n == 0: maximum area of entry (worst possible score)
    # - else: n - 1 (for calibration purposes)
    # Remember: Golf rules; lower is better.
    scores = tf.where(
        condition = tf.math.equal(sums, 0),
        x = tf.constant(data.shape[-2] * data.shape[-1], dtype=sums.dtype),
        y = tf.math.subtract(sums, 1)
    )
    # Calculate the final loss by dividing sum of scores over maximum scores.
    total = tf.math.reduce_sum(scores)
    shape = tf.shape(data, out_type=sums.dtype)
    area = tf.math.reduce_prod(shape)
    loss = tf.math.divide(total, area)
    return loss
    
simple_brevity(batch)

In [ ]:
a = totals.numpy()
print(a)
b = area.numpy()
print(b)
c = batch.shape
print(c)
#d = batch.shape[-3]
#print(d)
d = [-1, -2]
print(d)
e = tf.math.reduce_sum(tf.math.sign(batch), axis=d)
print(e)
f = tf.math.equal(e, 0)
print(f)
g = batch.shape[-2]*batch.shape[-1]
print(g)
h = tf.where(f, g, tf.math.subtract(e,1))
print(h)
i = tf.math.reduce_sum(h)
print(i)
j = tf.math.divide(i, b)
print(j)
#k = tf.math.reduce_prod(tf.where())
k = tf.math.mod(d, len(batch.shape))
print(k)

In [ ]:
temp = keras.losses.MeanAbsoluteError()

In [ ]:
temp(msg1, msg2)

---

In [ ]:
mask = tf.math.sign(msg)
mask

In [ ]:
tf.math.reduce_sum(mask, axis=-1)

---

In [ ]:
ts = cgael.LanguageTokenSet("CHAT", '-')
gen = SimpleColorGenerator([Swatch.WHITE, Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.BLUE, Swatch.YELLOW, Swatch.CYAN, Swatch.MAGENTA], blur=0, batch_lock=True)
model = SimpleColorModel(token_set=ts, word_count=1, word_length=3)

In [ ]:
ga_inst = model.train(
    generator = gen,
    generations = 100,
    num_solutions = 50,
    num_parents_mating = 5,
    mutation_percent_genes = .2,
)

In [ ]:
np.save(filepath, np.array(model.model.get_weights(), dtype="object"))

# Evaluation

In [ ]:
plot = ga_inst.plot_fitness(title=experiment_id, color="#0C69D3")
plot.savefig(os.path.join(root_folder, experiment_folder, f"{experiment_id}.png"))

In [ ]:
KEY_SWATCH = "swatch"
KEY_TEXT = "text"
KEY_IN = "input"
KEY_OUT = "output"
KEY_ROUND = "output_rounded"

swatches = [Swatch.BLACK, Swatch.RED, Swatch.GREEN, Swatch.YELLOW, Swatch.BLUE, Swatch.MAGENTA, Swatch.CYAN, Swatch.WHITE]
samples = np.array([[sample_swatch(x)] for x in swatches])
lang, out = model.model(samples)
text = [ts.decode(x) for x in lang]

d = {x:[] for x in [KEY_SWATCH, KEY_TEXT, KEY_IN, KEY_OUT, KEY_ROUND]}
for s, t, i, o in zip(swatches, text, samples, out):
    d[KEY_SWATCH].append(s)
    d[KEY_TEXT].append(t)
    d[KEY_IN].append(i)
    o = o.numpy()
    d[KEY_OUT].append(o)
    o = np.round(o)
    d[KEY_ROUND].append(o)
df = pd.DataFrame(data=d)
display(df)

In [ ]:
KEY_TEXT = "text"
KEY_OUT = "output"
KEY_ROUND = "output_rounded"

def generate_words(tokens, n):
    temp = tokens
    total = [''] + temp
    for _ in range(n-1):
        temp = [x+y for x in temp for y in tokens]
        total = total + temp
    return total

text = generate_words(ts.alphabet_tokens, 3)
data = np.array([ts.encode(x, shape=(1,3)) for x in text])
out = model.listener(data)

d = {x:[] for x in [KEY_TEXT, KEY_OUT, KEY_ROUND]}
for t, o in zip(text, out):
    d[KEY_TEXT].append(t)
    o = o.numpy()
    d[KEY_OUT].append(o)
    o = np.round(o)
    d[KEY_ROUND].append(o)
df = pd.DataFrame(data=d)
display(df)